# Fake_News_Flask_Application

**Installing the Libraries**

In [ ]:
!pip install flask flask-cors pyngrok transformers tensorflow

*Building the Flask Model for frontend integration of the trained BERT uncased Model **

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import os
import threading
from pyngrok import ngrok
import time
from google.colab import userdata # Import userdata to access secrets

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Global variables
MAX_LEN = 512
model = None
tokenizer = None
model_loaded = False
# Set a default port, can be changed later
PORT = 5000


def regular_encode(texts, tokenizer, maxlen=MAX_LEN):
    """Encode texts using the tokenizer"""
    enc_di = tokenizer.batch_encode_plus(
        texts,
        #return_attention_masks=False,
        #return_token_type_ids=False,
        #pad_to_max_length=True,
        max_length=maxlen,
        truncation=True
    )
    return np.array(enc_di['input_ids'])

def build_model(transformer, max_len=MAX_LEN):
    """Build the model architecture"""
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    x = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs=input_word_ids, outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

@app.route('/')
def index():
    return '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Fake News Detector - Colab</title>
        <style>
            body { font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }
            .container { background: #f8f9fa; padding: 30px; border-radius: 15px; box-shadow: 0 0 20px rgba(0,0,0,0.1); }
            h1 { color: #333; text-align: center; }
            textarea { width: 100%; height: 150px; margin: 10px 0; padding: 10px; border: 2px solid #ddd; border-radius: 8px; }
            button { background: #007bff; color: white; padding: 12px 24px; border: none; border-radius: 8px; cursor: pointer; margin: 5px; }
            button:hover { background: #0056b3; }
            button:disabled { background: #ccc; cursor: not-allowed; }
            .result { margin-top: 20px; padding: 20px; border-radius: 8px; font-weight: bold; }
            .fake { background: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; }
            .real { background: #d4edda; color: #155724; border: 2px solid #c3e6cb; }
            .status { padding: 10px; margin: 10px 0; border-radius: 5px; }
            .loading { background: #fff3cd; color: #856404; }
            .success { background: #d4edda; color: #155724; }
            .error { background: #f8d7da; color: #721c24; }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>🔍 Fake News Detector</h1>
            <p><strong>Model Status:</strong> <span id="status">''' + ('Loaded ✅' if model_loaded else 'Not Loaded ❌') + '''</span></p>

            <div>
                <h3>📝 Enter News Text:</h3>
                <textarea id="textInput" placeholder="Paste your news article here..."></textarea>
                <br>
                <button onclick="predictText()" id="predictBtn" ''' + ('>' if model_loaded else 'disabled>') + '''Analyze Text</button>
                <button onclick="clearText()">Clear</button>
            </div>

            <div id="result"></div>
        </div>

        <script>
            async function predictText() {
                const text = document.getElementById('textInput').value.trim();
                if (!text) {
                    alert('Please enter some text to analyze');
                    return;
                }

                const resultDiv = document.getElementById('result');
                resultDiv.innerHTML = '<div class="status loading">🔄 Analyzing text...</div>';

                try {
                    const response = await fetch('/predict', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({text: text})
                    });

                    const result = await response.json();

                    if (result.error) {
                        resultDiv.innerHTML = '<div class="status error">❌ Error: ' + result.error + '</div>';
                        return;
                    }

                    const isReal = result.prediction === 'real';
                    resultDiv.innerHTML = `
                        <div class="result ${isReal ? 'real' : 'fake'}">
                            <h3>${isReal ? '✅ Likely REAL News' : '⚠️ Potentially FAKE News'}</h3>
                            <p><strong>Confidence:</strong> ${result.confidence.toFixed(1)}%</p>
                            <p><strong>Real Probability:</strong> ${(result.probabilities.real * 100).toFixed(1)}%</p>
                            <p><strong>Fake Probability:</strong> ${(result.probabilities.fake * 100).toFixed(1)}%</p>
                        </div>
                    `;
                } catch (error) {
                    resultDiv.innerHTML = '<div class="status error">❌ Error: ' + error.message + '</div>';
                }
            }

            function clearText() {
                document.getElementById('textInput').value = '';
                document.getElementById('result').innerHTML = '';
            }
        </script>
    </body>
    </html>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    global model, tokenizer

    if not model_loaded or model is None or tokenizer is None:
        return jsonify({'error': 'Model not loaded. Please load the model first.'})

    try:
        data = request.json
        text = data.get('text', '').strip()

        if not text:
            return jsonify({'error': 'No text provided'})

        # Encode text
        encoded_text = regular_encode([text], tokenizer, maxlen=MAX_LEN)

        # Make prediction
        with app.app_context():
            prediction = model.predict(encoded_text, verbose=0)

        # Get probabilities
        real_prob = float(prediction[0][0])
        fake_prob = float(prediction[0][1])

        # Determine result
        is_real = real_prob > fake_prob
        confidence = max(real_prob, fake_prob) * 100

        return jsonify({
            'prediction': 'real' if is_real else 'fake',
            'confidence': confidence,
            'probabilities': {
                'real': real_prob,
                'fake': fake_prob
            }
        })

    except Exception as e:
        return jsonify({'error': f'Prediction error: {str(e)}'})

@app.route('/status')
def status():
    return jsonify({
        'model_loaded': model_loaded,
        'model_available': model is not None,
        'tokenizer_available': tokenizer is not None
    })

def load_model_function(model_path, model_name="bert-base-uncased"):
    """Function to load the model - call this from your notebook"""
    global model, tokenizer, model_loaded

    try:
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        print("Loading transformer...")
        transformer_layer = TFAutoModel.from_pretrained(model_name)

        print("Building model architecture...")
        model = build_model(transformer_layer, max_len=MAX_LEN)

        print(f"Loading weights from {model_path}...")
        model.load_weights(model_path)

        model_loaded = True
        print("✅ Model loaded successfully!")
        return True

    except Exception as e:
        print(f"❌ Error loading model: {str(e)}")
        model_loaded = False
        return False

def run_flask():
    """Function to run Flask in a separate thread"""
    # Use the global PORT variable
    app.run(host='0.0.0.0', port=PORT, debug=False, use_reloader=False)

def start_server(port=PORT, ngrok_token=None):
    """Start the Flask server with ngrok tunnel"""
    global PORT
    PORT = port # Set the global PORT variable

    # Authenticate ngrok using the provided token or a secret
    if ngrok_token:
        try:
            ngrok.set_auth_token(ngrok_token)
            print("ngrok authtoken set from argument.")
        except Exception as e:
            print(f"Error setting ngrok authtoken from argument: {e}")
    else:
        # Make sure you have your ngrok authtoken stored in Colab secrets as 'NGROK_AUTH_TOKEN'
        try:
            ngrok_token_from_secrets = userdata.get('NGROK_AUTH_TOKEN')
            if ngrok_token_from_secrets:
                ngrok.set_auth_token(ngrok_token_from_secrets)
                print("ngrok authtoken set from Colab secrets.")
            else:
                print("NGROK_AUTH_TOKEN not found in Colab secrets and no token provided as argument. ngrok tunnel may not work.")
        except Exception as e:
            print(f"Error setting ngrok authtoken from secrets: {e}")


    # Start Flask in a separate thread
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.daemon = True
    flask_thread.start()

    # Wait a moment for Flask to start
    time.sleep(3)

    try:
        # Create ngrok tunnel
        public_url = ngrok.connect(PORT)
        print(f"🌐 Public URL: {public_url}")
        print(f"🔗 Access your app at: {public_url}")
        return public_url
    except Exception as e:
        print(f"❌ Error creating ngrok tunnel: {str(e)}")
        print("Please check if your ngrok authtoken is valid and added to Colab secrets or provided as an argument.")
        print("Also, try running start_server() with a different port number, e.g., start_server(5001)")
        return None

# /content/drive/MyDrive/Ensemble_model/fold-0.h5

**Loading the Saved trained Model from Drive**

In [ ]:
load_model_function('/content/drive/MyDrive/Ensemble_model/fold-0.h5', 'bert-base-uncased')

Loading tokenizer...
Loading transformer...


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

Building model architecture...


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

Loading weights from /content/drive/MyDrive/Ensemble_model/fold-0.h5...


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

✅ Model loaded successfully!


True

In [ ]:
start_server()

Error setting ngrok authtoken: Secret NGROK_AUTH_TOKEN does not exist.
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected excepti

❌ Error creating ngrok tunnel: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.
Please check if your ngrok authtoken is valid and added to Colab secrets.
Also, try running start_server() with a different port number, e.g., start_server(5001)


**Starting the Ngrok Server to host the Flask Application**

In [ ]:
start_server(port=5001, ngrok_token="2x6FSAApBV4khWxkezSQZLysu8v_6r6bmNncrkqNvjQ3nLFrH")

ngrok authtoken set from argument.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an a

🌐 Public URL: NgrokTunnel: "https://f6d5-34-53-114-36.ngrok-free.app" -> "http://localhost:5001"
🔗 Access your app at: NgrokTunnel: "https://f6d5-34-53-114-36.ngrok-free.app" -> "http://localhost:5001"


<NgrokTunnel: "https://f6d5-34-53-114-36.ngrok-free.app" -> "http://localhost:5001">


1. First, load your model:
   load_model_function('/path/to/your/fold-0.h5', 'bert-base-uncased')

2. Then start the server:
   start_server()

3. Access the public URL that will be displayed

Example:
   load_model_function('/content/fold-0.h5', 'bert-base-uncased')
   url = start_server()
